<a href="https://colab.research.google.com/github/Luna0216/2025-DL-final-project/blob/main/Llmma_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bitsandbytes
!pip install transformers accelerate peft
!pip install einops
!pip install -q peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 115.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
!pip install -U bitsandbytes

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `0524Llmma` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `0524Llmm

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 8,396,800 || all params: 6,615,748,608 || trainable%: 0.1269


In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import torch
from sklearn.model_selection import train_test_split

In [ ]:
import string
import re
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)
def remove_20(text):
  space = re.compile(r'%20')
  return space.sub(r' ', text)

def remove_atsymbol(text):
    name = re.compile(r'@\S+')
    return name.sub(r' atsymbol ', text)


def remove_hashtag(text):
    hashtag = re.compile(r'#')
    return hashtag.sub(r' hashtag ', text)


def remove_exclamation(text):
    exclamation = re.compile(r'!')
    return exclamation.sub(r' exclamation ', text)


def remove_question(text):
    question = re.compile(r'?')
    return question.sub(r' question ', text)


def remove_punc(text):
    return text.translate(str.maketrans('','',string.punctuation))


def remove_number(text):
    number = re.compile(r'\d+')
    return number.sub(r' number ', text)


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               u"\x89û"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' emoji ', string)




In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv("test.csv")[['id', 'text']]

#參考https://www.kaggle.com/code/masatoshikato/effective-eda-bert-in-nlp-bert-baseline，清除讓資料剩下純文字
train_df['text'] = train_df['text'].apply(lambda text: remove_html(text))
train_df['text'] = train_df['text'].apply(lambda text: remove_atsymbol(text))
train_df['text'] = train_df['text'].apply(lambda text: remove_hashtag(text))
train_df['text'] = train_df['text'].apply(lambda text: remove_exclamation(text))
train_df['text'] = train_df['text'].apply(lambda text: remove_punc(text))
train_df['text'] = train_df['text'].apply(lambda text: remove_number(text))
train_df['text'] = train_df['text'].apply(lambda text: remove_emoji(text))
train_df['text'] = train_df['text'].apply(lambda text: remove_20(text))

test_df['text'] = test_df['text'].apply(lambda text: remove_html(text))
test_df['text'] = test_df['text'].apply(lambda text: remove_atsymbol(text))
test_df['text'] = test_df['text'].apply(lambda text: remove_hashtag(text))
test_df['text'] = test_df['text'].apply(lambda text: remove_exclamation(text))
test_df['text'] = test_df['text'].apply(lambda text: remove_punc(text))
test_df['text'] = test_df['text'].apply(lambda text: remove_number(text))
test_df['text'] = test_df['text'].apply(lambda text: remove_emoji(text))
test_df['text'] = test_df['text'].apply(lambda text: remove_20(text))





#參考https://www.kaggle.com/code/divyadevadas/nlp-tweet-bert-roberta-eda-tensorflow-pytorch
train_df['text'] = train_df['text'].apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(x)))
test_df['text'] = test_df['text'].apply(lambda x: re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(x)))

In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)



from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model


model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto"
)
train_df = train_df[['text', 'target']]

train_split, val_split = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['target'])
train_dataset = Dataset.from_pandas(train_split.reset_index(drop=True))
val_dataset   = Dataset.from_pandas(val_split.reset_index(drop=True))
test_dataset  = Dataset.from_pandas(test_df.reset_index(drop=True))
train_dataset = train_dataset.rename_column("target", "label")
val_dataset = val_dataset.rename_column("target", "label")

def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, padding="max_length")

train_dataset = train_dataset.map(tokenize_fn, batched=True)
val_dataset   = val_dataset.map(tokenize_fn, batched=True)
tokenized_test = test_dataset.map(tokenize_fn, batched=True)
train_dataset = train_dataset.remove_columns(["text"])
val_dataset = val_dataset.remove_columns(["text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
model.base_model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 22.7 MB/s eta 0:00:00


In [ ]:
def objective(trial):
    import os

    # 搜尋 LoRA 超參數
    r = trial.suggest_categorical("r", [8, 16, 32, 64])
    alpha = trial.suggest_int("lora_alpha", 8, 64)
    dropout = trial.suggest_float("lora_dropout", 0.0, 0.3)

    # 搜尋訓練超參數
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-4)
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_train_epochs = trial.suggest_int("num_train_epochs", 1, 4)

    # LoRA 設定
    lora_config = LoraConfig(
        r=r,
        lora_alpha=alpha,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=dropout,
        bias="none",
        task_type="SEQ_CLS"
    )

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
        quantization_config=bnb_config,
        device_map="auto"
    )
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)

    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.pad_token_id
    model.base_model.config.pad_token_id = tokenizer.pad_token_id

    # 訓練參數
    training_args = TrainingArguments(
        output_dir=f"./results/trial_{trial.number}",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        eval_strategy="epoch",
        save_strategy="no",
        logging_dir=f"./logs/trial_{trial.number}",
        learning_rate=learning_rate,
        disable_tqdm=False,
        fp16=True
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
    )

    trainer.train()
    metrics = trainer.evaluate()

    # 儲存每個 trial 的模型與 tokenizer
    save_path = f"./results/trial_{trial.number}"
    model.save_pretrained(f"{save_path}/model")
    tokenizer.save_pretrained(f"{save_path}/tokenizer")

    print(f"Trial {trial.number} 儲存於 {save_path}")
    return metrics["eval_loss"]


In [ ]:
import optuna

study = optuna.create_study(direction="minimize")

study.optimize(objective, n_trials=10)

[I 2025-05-26 09:42:04,438] A new study created in memory with name: no-name-48f429eb-fc1f-486e-968d-d29cfbe3d00b
<ipython-input-20-05a08fe12127>:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-6, 5e-4)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-20-05a08fe12127>:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False

Epoch,Training Loss,Validation Loss
1,0.640700,0.408645
2,0.364400,0.418900


[I 2025-05-26 09:54:03,150] Trial 0 finished with value: 0.41890043020248413 and parameters: {'r': 8, 'lora_alpha': 22, 'lora_dropout': 0.08467526748524153, 'learning_rate': 6.866074583481234e-05, 'batch_size': 8, 'num_train_epochs': 2}. Best is trial 0 with value: 0.41890043020248413.


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-20-05a08fe12127>:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False

Epoch,Training Loss,Validation Loss
1,0.690600,0.668156


[I 2025-05-26 12:44:26,581] Trial 9 finished with value: 0.6681559681892395 and parameters: {'r': 64, 'lora_alpha': 59, 'lora_dropout': 0.2811433656354753, 'learning_rate': 0.00035610309136496835, 'batch_size': 4, 'num_train_epochs': 1}. Best is trial 0 with value: 0.41890043020248413.


Trial 9 儲存於 ./results/trial_9


In [ ]:
import json

with open("best_params.json", "w") as f:
    json.dump(study.best_params, f)

print("最佳參數已儲存為 best_params.json")


最佳參數已儲存為 best_params.json


In [ ]:
model.print_trainable_parameters()

In [ ]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
print(train_dataset[0])

{'label': 1, 'input_ids': [1, 317, 465, 29891, 4272, 7826, 4234, 298, 2960, 851, 392, 287, 297, 4116, 554, 29891, 18204, 15007, 303, 555, 29871, 756, 400, 351, 319, 29877, 4345, 1143, 415, 535, 29895, 29968, 29883, 29911, 698, 29879, 29928, 1353, 259, 756, 400, 351, 474, 2909, 417, 345, 29871, 756, 400, 351, 3143, 17079], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
import json
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding, EarlyStoppingCallback
import wandb

# 載入最佳超參數
with open("best_params.json", "r") as f:
    best_params = json.load(f)

# LoRA 設定
lora_config = LoraConfig(
    r=best_params["r"],
    lora_alpha=best_params["lora_alpha"],
    target_modules=["q_proj", "v_proj"],
    lora_dropout=best_params["lora_dropout"],
    bias="none",
    task_type="SEQ_CLS"
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto"
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

# tokenizer 處理
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
model.base_model.config.pad_token_id = tokenizer.pad_token_id

# 初始化 wandb
wandb.init(project="llama-binclass", name="run_llama_4bit_best")

# 訓練參數套用 best_params
training_args = TrainingArguments(
    output_dir="./llama-binclass",
    per_device_train_batch_size=best_params["batch_size"],
    per_device_eval_batch_size=best_params["batch_size"],
    num_train_epochs=best_params["num_train_epochs"],

    eval_strategy="steps",
    eval_steps=100,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    learning_rate=best_params["learning_rate"],
    report_to="wandb",
    run_name="run_llama_4bit_best",
    remove_unused_columns=False,
    logging_dir="./logs"
)

# 建立 Trainer 並訓練
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<ipython-input-27-bca0158a9114>:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
100,0.556500,0.654447
200,0.480500,0.529504
300,0.598300,0.468972
400,0.453600,0.451686
500,0.314800,0.447558
600,0.502800,0.421861
700,0.495400,0.407116
800,0.411300,0.426202
900,0.322500,0.425112
1000,0.408400,0.412613


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=1524, training_loss=0.4615712089175627, metrics={'train_runtime': 1038.6905, 'train_samples_per_second': 11.726, 'train_steps_per_second': 1.467, 'total_flos': 2.5390374135496704e+16, 'train_loss': 0.4615712089175627, 'epoch': 2.0})

In [ ]:

raw_test_df = pd.read_csv("test.csv")[["id", "text"]]

test_dataset = Dataset.from_pandas(raw_test_df)

tokenized_test = test_dataset.map(tokenize_fn, batched=True)


tokenized_test_for_model = tokenized_test.remove_columns(["id", "text"])
tokenized_test = tokenized_test.remove_columns(["text", "id"])
predictions = trainer.predict(tokenized_test)
pred_labels = predictions.predictions.argmax(axis=1)

test_df["target"] = pred_labels

test_df= test_df[["id","target"]]
test_df.to_csv("llama_r_16_cleaning.csv", index=False)

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]